In [ ]:
import os
import math
import pandas as pd
import requests
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont, ImageFilter
from sqlalchemy import create_engine

# === CONFIG ===
TMDB_API_KEY    = "1dc3598610c5a9627425e69414f22ce2"
TMDB_FIND_URL   = "https://api.themoviedb.org/3/find/"
TMDB_IMG_BASE   = "https://image.tmdb.org/t/p/original"
DB_NAME         = "imdb"
SQL_PATH        = r"C:\Users\mason\Documents\last_10_Bafta_scripted_comedy_winner.sql"

# === LAYOUT PARAMETERS ===
POSTER_HEIGHT    = 1200       # bump up poster height for max resolution
THUMB_SPACE      = 0          # no vertical gap between nominee thumbs
ROW_GAP          = 0          # no gap between rows
YEAR_FONT_SIZE   = 96         # scale with poster height
HEADER_H         = 240        # larger header for high-res
HEADER_FONT_SIZE = 120
SUB_FONT_SIZE    = 60
SHADOW_OFFSET    = (5, 5)
BLUR_RADIUS      = 5

# === LOAD DATA ===
engine = create_engine(
    f"mssql+pyodbc://@localhost/{DB_NAME}"
    "?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)
with open(SQL_PATH, "r", encoding="utf-8") as f:
    sql = f.read()
df = pd.read_sql(sql, engine)
df["status"] = df["bafta_status"]

years = sorted(df["awardyear"].unique())[-10:]
df    = df[df["awardyear"].isin(years)]
wins  = df[df.status == "winner"].set_index("awardyear")
noms  = df[df.status == "nominee"]

# === FETCH & PAD POSTERS ===
def fetch_and_pad(imdb_id, height):
    # highest-res 'original' size from TMDb, then scale to `height`
    r = requests.get(
        TMDB_FIND_URL + imdb_id,
        params={"api_key": TMDB_API_KEY, "external_source": "imdb_id"}
    ).json()
    res = r.get("tv_results", []) + r.get("movie_results", [])
    if not res or not res[0].get("poster_path"):
        return None
    img = Image.open(BytesIO(requests.get(
        TMDB_IMG_BASE + res[0]["poster_path"]
    ).content)).convert("RGBA")
    # scale to desired height (maintains high quality)
    scale = height / img.height
    w = int(img.width * scale)
    return img.resize((w, height), Image.LANCZOS)

# preload
posters = {yr: fetch_and_pad(wins.loc[yr,"tconst"], POSTER_HEIGHT) for yr in years}

# === THUMBNAILS & CELL W ===
thumbs   = {}
cell_w   = {}
for yr in years:
    ids = list(noms[noms.awardyear == yr]["tconst"])
    n   = len(ids)
    # scale nominees to fill poster height
    h   = (POSTER_HEIGHT - (n-1)*THUMB_SPACE)//n if n else 0
    ts  = [fetch_and_pad(tc, h) for tc in ids]
    ts  = [t for t in ts if t]
    thumbs[yr] = ts
    mw = max((t.width for t in ts), default=0)
    pw = posters[yr].width if posters[yr] else 0
    cell_w[yr] = pw + mw

# === CANVAS ===
grid     = [years[:5], years[5:]]
row_w    = [sum(cell_w[y] for y in row) for row in grid]
mont_w   = max(row_w)
mont_h   = 2 * POSTER_HEIGHT
canvas_w = mont_w
canvas_h = HEADER_H + mont_h
canvas   = Image.new("RGBA", (canvas_w, canvas_h), (40,40,40,255))
draw     = ImageDraw.Draw(canvas)

# === FONTS ===
try:
    header_f   = ImageFont.truetype("arialbd.ttf", HEADER_FONT_SIZE)
    sub_f      = ImageFont.truetype("arial.ttf", SUB_FONT_SIZE)
    year_f     = ImageFont.truetype("arialbd.ttf", YEAR_FONT_SIZE)
except IOError:
    header_f = sub_f = year_f = ImageFont.load_default()

# === HEADER BAR ===
title = f"BAFTA Scripted Comedy Awards: Winners & Nominees ({years[0]}–{years[-1]})"
tb    = draw.textbbox((0,0), title, font=header_f)
tx    = (canvas_w - (tb[2]-tb[0]))//2
ty    = (HEADER_H - (tb[3]-tb[1]) - SUB_FONT_SIZE)//2
draw.text((tx, ty), title, font=header_f, fill="white")

sub   = "@relaxedmason   •   Data sources: IMDb • TMDb • Wikipedia"
sb    = draw.textbbox((0,0), sub, font=sub_f)
sx    = (canvas_w - (sb[2]-sb[0]))//2
sy    = ty + (tb[3]-tb[1]) + 10
draw.text((sx, sy), sub, font=sub_f, fill=(200,200,200))

# === MONTAGE ===
for r_i, row in enumerate(grid):
    y0 = HEADER_H + r_i * POSTER_HEIGHT
    x0 = 0
    for yr in row:
        p = posters[yr]
        if p is None:
            x0 += cell_w[yr]
            continue

        # shadow
        mask    = Image.new("L", p.size, 255)
        shadow  = mask.filter(ImageFilter.GaussianBlur(BLUR_RADIUS))
        sh_col  = Image.new("RGBA", p.size, (0,0,0,150))
        canvas.paste(sh_col, (x0+SHADOW_OFFSET[0], y0+SHADOW_OFFSET[1]), shadow)

        # poster
        canvas.paste(p, (x0, y0), p)

        # year label
        draw.text((x0+10, y0+10), str(yr), font=year_f,
                  fill="white", stroke_width=3, stroke_fill="black")

        # nominees (centered)
        ts      = thumbs[yr]
        total_h = sum(t.height for t in ts) + THUMB_SPACE*(len(ts)-1)
        tyt     = y0 + (POSTER_HEIGHT - total_h)//2
        txt_x   = x0 + p.width
        for t in ts:
            canvas.paste(t, (txt_x, tyt), t)
            tyt += t.height + THUMB_SPACE

        x0 += cell_w[yr]

# === SAVE HIGH-RES OUTPUT ===
out = r"C:\Users\mason\Documents\bafta_scripted_two_row_montage_hires.png"
canvas.convert("RGB").save(out, dpi=(300,300))
print("✅ Saved high-res montage to", out)